# **Extract, Transform, Load (ETL) Process**

## Objectives

* The objective of this notebook is to perform an ETL process of the data I have gathered from Kaggle.
* I have fetched the data which you can find in the datasets/raw folder.
* There are 4 related csv files that contain foreign keys that relate to eachother such as player_id, match_id and tournament_id. I will use these keys to clean and merge the data.
## Inputs

* My inputs are 4 csv files:
    * matches.csv
    * players.csv
    * scores.csv
    * tournaments.csv

## Outputs

* I am going to output one main table. From this table I will filter information into smaller csv files where I can work on them more easily to create visualisations.

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir


'/Users/stephenbeese/GitHub/Snooker-Data-Analysis/Snooker-Data-Analysis/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/stephenbeese/GitHub/Snooker-Data-Analysis/Snooker-Data-Analysis'

# Imports

To clean and merge this data I will first need to import some libraries

In [ ]:
import pandas as pd

# Set up data directories

In [5]:
# Set the file path for raw data
raw_data_dir = os.path.join(current_dir, 'datasets/raw')

# Set the file path for clean data
clean_data_dir = os.path.join(current_dir, 'datasets/clean')

# Section 1 - matches.csv

In [6]:
# Load all csv files into Pandas DataFrames
df_matches = pd.read_csv(os.path.join(raw_data_dir, 'matches.csv'))
df_players = pd.read_csv(os.path.join(raw_data_dir, 'players.csv'))
df_scores = pd.read_csv(os.path.join(raw_data_dir, 'scores.csv'))
df_tournaments = pd.read_csv(os.path.join(raw_data_dir, 'tournaments.csv'))


---

In [7]:
# Display the head of df_matches
df_matches.head()

,tournament_id,match_id,date,stage,best_of,player1_name,player1_url,player2_name,player2_url,score1,score2,frames_scores,is_walkover
0,753,82716,NaN,Final,31,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Alex Higgins,https://cuetracker.net/players/alex-higgins,16,15,20-58; 31-90; 56-52; 26-87(67); 0-114(67); 73(...,False
1,753,82718,NaN,Semi-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Tony Meo,https://cuetracker.net/players/tony-meo,9,7,71-8; 50-71(55); 31-62; 69-30; 73-61; 34-77(52...,False
2,753,82717,NaN,Semi-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,Ray Reardon,https://cuetracker.net/players/ray-reardon,9,6,28-71; 67(50)-29; 74(74)-0; 53-79; 60-54; 112(...,False
3,753,82721,NaN,Quarter-final,17,Terry Griffiths,https://cuetracker.net/players/terry-griffiths,Steve Davis,https://cuetracker.net/players/steve-davis,9,6,"1-103; 117(60,57)-6; 5-105(60); 57-60; 79-0; 2...",False
4,753,82719,NaN,Quarter-final,17,Alex Higgins,https://cuetracker.net/players/alex-higgins,John Spencer,https://cuetracker.net/players/john-spencer,9,5,69(54)-31; 103-21; 72-48; 33-82; 40-56; 71-51;...,False


In [8]:
# Check missing values in the dataframe
df_matches.isnull().sum()

tournament_id         0
match_id              0
date             138711
stage                 0
best_of               0
player1_name          0
player1_url           0
player2_name          0
player2_url           0
score1                0
score2                0
frames_scores    123278
is_walkover           0
dtype: int64

From the code above we can see that there are a lot of missing values in both the `date` column and `frames_scores` column.

As we are not interested in `frames_scores` we can remove this column completely.

The `date` column however is useful to us, luckily in the `tournaments.csv` file it contains both the `season` and `year` of each tournament. We can use the `tournament_id` to get this information and create a new column in the `matches.csv` file.

We will do this later but for now we can drop the `date` and `frames_scores` as they are not necessary.

Looking further into the DataFrame, there are some more unecessary columns that won't be needed for the analysis we are looking at in this project.
* `player1_url`
* `player2_url`
* `is_walkover`

In [9]:
# Create a copy of df_matches
df_matches = df_matches.copy()

# Delete date and frames_scores from df_matches
df_matches = df_matches.drop(columns=['date', 'frames_scores', 'player1_url', 'player2_url', 'is_walkover'])

In [10]:
# Display the head of df_matches after cleaning
df_matches.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,score1,score2
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5


In [11]:
# Check missing values in the dataframe
df_matches.isnull().sum()

tournament_id    0
match_id         0
stage            0
best_of          0
player1_name     0
player2_name     0
score1           0
score2           0
dtype: int64

In [12]:
# Check for duplicate matches
df_matches[df_matches.duplicated()]

,tournament_id,match_id,stage,best_of,player1_name,player2_name,score1,score2


As you can see from the code cell above we now have a cleaned `matches.csv` dataset with no null values or duplicates.

In the next cell I will save this to a new .csv file and continue working on the other dataframes.

In [13]:
# Save cleaned DataFrame to a new .csv file
df_matches.to_csv(os.path.join(clean_data_dir, 'matches_cleaned.csv'), index=False)

# Section 2 - players.csv

Section 2 content

In [14]:
# Display the head of df_players
df_players.head()

,url,id,first_name,last_name,full_name,country
0,https://cuetracker.net/players/mohammed-a-belg...,mohammed-a-belgaizi,Mohammed,A Belgaizi,Mohammed A Belgaizi,United Arab Emirates
1,https://cuetracker.net/players/ishaq-a-khaleg,ishaq-a-khaleg,Ishaq,A Khaleg,Ishaq A Khaleg,Bahrain
2,https://cuetracker.net/players/ahmed-a-asere,ahmed-a-asere,Ahmed,A. Asere,Ahmed A. Asere,Saudi Arabia
3,https://cuetracker.net/players/magnus-aagaard,magnus-aagaard,Magnus,Aagaard,Magnus Aagaard,Denmark
4,https://cuetracker.net/players/asbjorn-aalberg,asbjorn-aalberg,Asbjorn,Aalberg,Asbjorn Aalberg,Norway


In [15]:
# Check for missing values
df_players.isnull().sum()

url           0
id            0
first_name    0
last_name     0
full_name     0
country       0
dtype: int64

In [16]:
# Check for duplicate players
df_players[df_players.duplicated()]

,url,id,first_name,last_name,full_name,country


As you can see from the code above there are no missing values or duplicates in this table.

This means we can look at the columns and decide what is useful to our project outcomes.

As we are looking at international players we are interested in the country and the player's names.

This means we can drop the player_url as that isn't crucial for our analysis.

I am going to drop this column and then save it to a new `.csv` file.

In [17]:
# Create a copy of df_players
df_players = df_players.copy()

# Drop the player_url column
df_players = df_players.drop(columns=['url'])

# Display the head of df_players
df_players.head()

,id,first_name,last_name,full_name,country
0,mohammed-a-belgaizi,Mohammed,A Belgaizi,Mohammed A Belgaizi,United Arab Emirates
1,ishaq-a-khaleg,Ishaq,A Khaleg,Ishaq A Khaleg,Bahrain
2,ahmed-a-asere,Ahmed,A. Asere,Ahmed A. Asere,Saudi Arabia
3,magnus-aagaard,Magnus,Aagaard,Magnus Aagaard,Denmark
4,asbjorn-aalberg,Asbjorn,Aalberg,Asbjorn Aalberg,Norway


In [18]:
# Save df_players to a CSV file
df_players.to_csv(os.path.join(clean_data_dir, 'players_cleaned.csv'), index=False)

---

# Section 3 - tournaments.csv

In [ ]:
# Show the head of df_tournaments
df_tournaments.head()

,id,season,year,name,full_name,url,status,category,prize,country,city
0,753,1982-1983,1982,UK Championship,1982 UK Championship,https://cuetracker.net/tournaments/uk-champion...,Professional,Non-ranking,47000.0,England,Preston
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,https://cuetracker.net/tournaments/world-amate...,Amateur,World Event,0.0,Canada,Calgary
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,https://cuetracker.net/tournaments/professiona...,Professional,Ranking,31500.0,England,Birmingham
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,https://cuetracker.net/tournaments/pontins-aut...,Pro-am,Event,0.0,Wales,Prestatyn
4,754,1982-1983,1982,International Open,1982 International Open,https://cuetracker.net/tournaments/internation...,Professional,Ranking,73500.0,England,Derby
...,...,...,...,...,...,...,...,...,...,...,...
2715,2884,2018-2019,2019,Guernsey Amateur Championship,2019 Guernsey Amateur Championship,https://cuetracker.net/tournaments/guernsey-am...,Amateur,National Championship,0.0,Guernsey,Various
2716,3321,2019-2020,2020,English Amateur Championship,2020 English Amateur Championship,https://cuetracker.net/tournaments/english-ama...,Amateur,National Championship,0.0,England,Cheltenham
2717,3362,NaN,2020,3 Kings Open,2020 3 Kings Open,https://cuetracker.net/tournaments/3-kings-ope...,Pro-am,Event,0.0,Austria,Rankweil
2718,3357,2019-2020,2020,Singapore Amateur Championship,2020 Singapore Amateur Championship,https://cuetracker.net/tournaments/singapore-a...,Amateur,National Championship,0.0,Singapore,Singapore


As we are only looking at the professional tour, I am going to filter these to ensure we are only left with tournaments that took place on the professional tour.

In [ ]:
# Copy the df_tournaments DataFrame
df_tournaments = df_tournaments.copy()

# Display tournaments DataFrame
df_tournaments

,id,season,year,name,full_name,url,status,category,prize,country,city
0,753,1982-1983,1982,UK Championship,1982 UK Championship,https://cuetracker.net/tournaments/uk-champion...,Professional,Non-ranking,47000.0,England,Preston
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,https://cuetracker.net/tournaments/world-amate...,Amateur,World Event,0.0,Canada,Calgary
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,https://cuetracker.net/tournaments/professiona...,Professional,Ranking,31500.0,England,Birmingham
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,https://cuetracker.net/tournaments/pontins-aut...,Pro-am,Event,0.0,Wales,Prestatyn
4,754,1982-1983,1982,International Open,1982 International Open,https://cuetracker.net/tournaments/internation...,Professional,Ranking,73500.0,England,Derby
...,...,...,...,...,...,...,...,...,...,...,...
2715,2884,2018-2019,2019,Guernsey Amateur Championship,2019 Guernsey Amateur Championship,https://cuetracker.net/tournaments/guernsey-am...,Amateur,National Championship,0.0,Guernsey,Various
2716,3321,2019-2020,2020,English Amateur Championship,2020 English Amateur Championship,https://cuetracker.net/tournaments/english-ama...,Amateur,National Championship,0.0,England,Cheltenham
2717,3362,NaN,2020,3 Kings Open,2020 3 Kings Open,https://cuetracker.net/tournaments/3-kings-ope...,Pro-am,Event,0.0,Austria,Rankweil
2718,3357,2019-2020,2020,Singapore Amateur Championship,2020 Singapore Amateur Championship,https://cuetracker.net/tournaments/singapore-a...,Amateur,National Championship,0.0,Singapore,Singapore


In [21]:
# Check missing values in DataFrame
df_tournaments.isnull().sum()

id            0
season       15
year          0
name          0
full_name     0
url           0
status        0
category      0
prize         1
country      30
city         74
dtype: int64

After filtering this dataframe to only show Professional tournaments you can see that we have some null values.

The columns that contain null values are:
* `season`
* `country`
* `city`

As our project aim is regarding the international growth of the sport we are going to drop any tournaments that do not have a country associated to them

In [22]:
# Drop rows with null values in country column
df_tournaments = df_tournaments[df_tournaments['country'].notna()]

# Check missing values in DataFrame
df_tournaments.isnull().sum()


id            0
season       15
year          0
name          0
full_name     0
url           0
status        0
category      0
prize         1
country       0
city         74
dtype: int64

After removing rows with null country values we are now left with two rows that contain missing values.

* One row is missing a `season` value
* One row is missing a `city` value

In [23]:
# View rows with null in specific columns
rows_with_nulls = df_tournaments[df_tournaments.isna().any(axis=1)]

# Display rows with null values
rows_with_nulls

,id,season,year,name,full_name,url,status,category,prize,country,city
6,2444,1982-1983,1982,Australian Amateur Championship,1982 Australian Amateur Championship,https://cuetracker.net/tournaments/australian-...,Amateur,National Championship,0.0,Australia,NaN
7,2443,1982-1983,1982,Scottish Open Championship,1982 Scottish Open Championship,https://cuetracker.net/tournaments/scottish-op...,Pro-am,National Championship,0.0,Scotland,NaN
9,2549,1981-1982,1982,All-Ireland Amateur Championship,1982 All-Ireland Amateur Championship,https://cuetracker.net/tournaments/all-ireland...,Amateur,National Championship,0.0,Ireland,NaN
21,2548,1981-1982,1982,Indian Amateur Championship,1982 Indian Amateur Championship,https://cuetracker.net/tournaments/indian-amat...,Amateur,National Championship,0.0,India,NaN
190,2491,1985-1986,1986,Canadian Championship,1986 Canadian Championship,https://cuetracker.net/tournaments/canadian-ch...,Amateur,National Championship,0.0,Canada,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2566,2710,2017-2018,2018,Northern Ireland Amateur Championship,2018 Northern Ireland Amateur Championship,https://cuetracker.net/tournaments/northern-ir...,Amateur,National Championship,0.0,Northern Ireland,NaN
2567,2701,NaN,2018,Welsh Amateur Championship,2018 Welsh Amateur Championship,https://cuetracker.net/tournaments/welsh-amate...,Amateur,National Championship,0.0,Wales,NaN
2568,3220,NaN,2019,Albanian Amateur Championship,2019 Albanian Amateur Championship,https://cuetracker.net/tournaments/albanian-am...,Amateur,National Championship,0.0,Albania,Unknown
2599,3279,NaN,2019,Russian Amateur Championship,2019 Russian Amateur Championship,https://cuetracker.net/tournaments/russian-ama...,Amateur,National Championship,0.0,Russia,Unknown


To fix these missing values I am first going to remove the `city` column as it is not necessary for our analysis.

I will also fill in the season based on the `year` column and format the value in the way it is presented in the other columns (`YYYY-YYYY`). I have found some information regarding this tournament <a href="https://cuetracker.net/tournaments/ebsa-qualifying-tour-play-offs/2014/1044" target="_blank" rel="noopener">here</a>. It states that the tournament was played in the 2013-2014 season so I will update this cell with that information.

In [24]:
# Remove city column
df_tournaments = df_tournaments.drop(columns=['city'])

# Display DataFrame
df_tournaments

,id,season,year,name,full_name,url,status,category,prize,country
0,753,1982-1983,1982,UK Championship,1982 UK Championship,https://cuetracker.net/tournaments/uk-champion...,Professional,Non-ranking,47000.0,England
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,https://cuetracker.net/tournaments/world-amate...,Amateur,World Event,0.0,Canada
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,https://cuetracker.net/tournaments/professiona...,Professional,Ranking,31500.0,England
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,https://cuetracker.net/tournaments/pontins-aut...,Pro-am,Event,0.0,Wales
4,754,1982-1983,1982,International Open,1982 International Open,https://cuetracker.net/tournaments/internation...,Professional,Ranking,73500.0,England
...,...,...,...,...,...,...,...,...,...,...
2715,2884,2018-2019,2019,Guernsey Amateur Championship,2019 Guernsey Amateur Championship,https://cuetracker.net/tournaments/guernsey-am...,Amateur,National Championship,0.0,Guernsey
2716,3321,2019-2020,2020,English Amateur Championship,2020 English Amateur Championship,https://cuetracker.net/tournaments/english-ama...,Amateur,National Championship,0.0,England
2717,3362,NaN,2020,3 Kings Open,2020 3 Kings Open,https://cuetracker.net/tournaments/3-kings-ope...,Pro-am,Event,0.0,Austria
2718,3357,2019-2020,2020,Singapore Amateur Championship,2020 Singapore Amateur Championship,https://cuetracker.net/tournaments/singapore-a...,Amateur,National Championship,0.0,Singapore


In [25]:
# Fill in season based on row index
df_tournaments.loc[2005, "season"] = "2013/2014"

In [26]:
# Check missing values
df_tournaments.isnull().sum()

id            0
season       14
year          0
name          0
full_name     0
url           0
status        0
category      0
prize         1
country       0
dtype: int64

Now our DataFrame has no missing values we can look at getting rid of any unecessary columns.

Most of the data in this DataFrame is useful to us however we do not need the `url` column as it's not useful to our analysis so I will drop that next.

In [27]:
# Drop url column
df_tournaments = df_tournaments.drop(columns=['url'])

# Display DataFrame head
df_tournaments.head()

,id,season,year,name,full_name,status,category,prize,country
0,753,1982-1983,1982,UK Championship,1982 UK Championship,Professional,Non-ranking,47000.0,England
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,Amateur,World Event,0.0,Canada
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,Professional,Ranking,31500.0,England
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,Pro-am,Event,0.0,Wales
4,754,1982-1983,1982,International Open,1982 International Open,Professional,Ranking,73500.0,England


Now that our tournaments.csv file has been cleaned we can now save it to a new cleaned file

In [28]:
# Save DataFrame to CSV
df_tournaments.to_csv(os.path.join(clean_data_dir, 'tournaments_cleaned.csv'), index=False)


# Section 4 - scores.csv

* We may not need this table for analysis however I am going to clean it anyway incase we need any information from it later in the project.

In [29]:
# Show the head of the DataFrame
df_scores.head()

# Copy the DataFrame
df_scores_copy = df_scores.copy()


In [30]:
# Show missing values
df_scores.isnull().sum()

match_id                  0
frame                     0
player                    0
score                     0
50plus_breaks_str    757723
dtype: int64

As we can see from the cell above there were 757,723 rows that do not contain a value for `50plus_breaks_str`.

We can assume that if this cell reads as `Null` or `NaN` then we can assume that there were no breaks over 50.

Therefore we can update these cells with "0".

In [31]:
# Update missing 50plus_breaks_str
df_scores['50plus_breaks_str'] = df_scores['50plus_breaks_str'].fillna('0')

# Check missing values
df_scores.isnull().sum()

match_id             0
frame                0
player               0
score                0
50plus_breaks_str    0
dtype: int64

Seeing as there are now no missing values in this dataset we can now save it as a new csv file.

We are not going to drop any columns as this is all useful information to us.

In [32]:
# Save DataFrame to CSV
df_scores.to_csv(os.path.join(clean_data_dir, 'scores_cleaned.csv'), index=False)


# Merging Datasets

Now that each dataset has been cleaned we can start to merge them and create some meaningful tables that are useful for analysis.

In [33]:
# Recap of Cleaned DataFrames
df_matches = pd.read_csv(os.path.join(clean_data_dir, 'matches_cleaned.csv'))
df_players = pd.read_csv(os.path.join(clean_data_dir, 'players_cleaned.csv'))
df_tournaments = pd.read_csv(os.path.join(clean_data_dir, 'tournaments_cleaned.csv'))
df_scores = pd.read_csv(os.path.join(clean_data_dir, 'scores_cleaned.csv'))

### Plan

My plan for merging these tables is as follows:
* I am going to create **one master table** that holds all information about the matches
* The master table will have the following columns:

| Column Name                   | Description                        | Source                                                                                            |
| ----------------------------- | ---------------------------------- | ------------------------------------------------------------------------------------------------- |
| `Match_id`                    | Unique match identifier            | `matches_r.csv` → `match_id`                                                                      |
| `tournament_id`               | Tournament identifier              | `matches_r.csv` → `tournament_id` (FK to `tournaments.csv` → `id`)                                |
| `season`                      | Snooker season (e.g., 2013/2014)   | `tournaments.csv` → `season` *(derive from match `date` if missing)*                              |
| `year`                        | Tournament year                    | `tournaments.csv` → `year`                                                                        |
| `tournament_name`             | Official tournament name           | `tournaments.csv` → `full_name` *(fallback `name`)*                                               |
| `stage`                       | Tournament stage (`QF`, `SF`, `F`) | `matches_r.csv` → `stage` *(normalise values)*                                                    |
| `best_of`                     | Max frames in the match            | `matches_r.csv` → `best_of`                                                                       |
| `player1_name`                | First player’s name                | `matches_r.csv` → `player1_name`                                                                  |
| `player1_country`             | First player’s country             | `players_r.csv` → `Country` *(join on `matches_r.player1_url = players_r.url`)*                   |
| `player2_name`                | Second player’s name               | `matches_r.csv` → `player2_name`                                                                  |
| `player2_country`             | Second player’s country            | `players_r.csv` → `Country` *(join on `matches_r.player2_url = players_r.url`)*                   |
| `player1_score`               | Frames won by player 1             | `matches_r.csv` → `score1`                                                                        |
| `player2_score`               | Frames won by player 2             | `matches_r.csv` → `score2`                                                                        |
| `tournament_country`          | Host country                       | `tournaments.csv` → `country`                                                                     |
| `tournament_prize`            | Prize fund (nominal)               | `tournaments.csv` → `prize`                                                                       |
| `match_winner`                | Winner’s name                      | **Derived:** `score1 > score2 → player1_name` else `player2_name`                                 |
| `match_winner_country`        | Winner’s country                   | **Derived:** from `player1_country` / `player2_country` based on `match_winner`                   |
| `is_international_match`      | Is one player outside of the UK?   | **Derived:** `player1_country or player2_country != UK`.                                          |
| `is_international_tournament` | Event hosted outside the UK?       | **Derived:** `tournament_country != {England, Scotland, Wales, Northern Ireland}`                 |



I will use the `matches_cleaned.csv` as the starting point for this table as it contains a lot of the same data. I will just add to it to create the master table.


In [34]:
# Display the df_matches table
df_matches

,tournament_id,match_id,stage,best_of,player1_name,player2_name,score1,score2
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5
...,...,...,...,...,...,...,...,...
193525,3275,203168,Group 1,5,Neil Robertson,Luca Brecel,3,1
193526,3275,203174,Group 1,5,Neil Robertson,Jack Lisowski,3,1
193527,3275,203167,Group 1,5,Mark Selby,Luca Brecel,3,0
193528,3275,203169,Group 1,5,Mark Selby,Ryan Day,3,2


In [35]:
# Make a copy of df_matches
df_matches = df_matches.copy()

# Display head
df_matches.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,score1,score2
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5


In [36]:
# List tournament columns I would like to merge
tournament_columns_to_merge = ["id","season", "year", "name", "country", "prize", "status"]

# Select only the tournament columns
df_tournaments_subset = df_tournaments[tournament_columns_to_merge]

# Display head
df_tournaments_subset.head()

,id,season,year,name,country,prize,status
0,753,1982-1983,1982,UK Championship,England,47000.0,Professional
1,1140,1982-1983,1982,World Amateur Championship - Men,Canada,0.0,Amateur
2,762,1982-1983,1982,Professional Players Tournament,England,31500.0,Professional
3,2586,1982-1983,1982,Pontins Autumn Open,Wales,0.0,Pro-am
4,754,1982-1983,1982,International Open,England,73500.0,Professional


Now that we have a subset of the tournaments DataFrame, we can look at merging this with the matches DataFrame by the tournament_id.

In [37]:
# Merge tournament and match data
df_master = (
    df_matches
    .merge(df_tournaments_subset, left_on='tournament_id', right_on='id', how='left', validate='many_to_one')
    .rename(columns={
        "name": "tournament_name",
        "country": "tournament_country",
        "prize": "tournament_prize_pool",
        "status": "tournament_status",
        "score1": "player1_score",
        "score2": "player2_score"
    })
)

# Remove duplicate tournament_id column
df_master.drop(columns=['id'], inplace=True)

# Display head
df_master.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982.0,UK Championship,England,47000.0,Professional
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982.0,UK Championship,England,47000.0,Professional
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982.0,UK Championship,England,47000.0,Professional


Now that we have merged the tournament and matches dataframes into a master. We can now look at querying the player's country and adding that tour our master DataFrame

In [38]:
# Merge player1_country using player1_name and full_name
df_master = df_master.merge(
    df_players[['full_name', 'country']].rename(columns={'full_name': 'player1_name', 'country': 'player1_country'}),
    on='player1_name',
    how='left'
)

# Merge player2_country using player2_name and full_name
df_master = df_master.merge(
    df_players[['full_name', 'country']].rename(columns={'full_name': 'player2_name', 'country': 'player2_country'}),
    on='player2_name',
    how='left'
)

# Display head
df_master.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,Northern Ireland
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,Wales
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,England


Now that we have all the data readily available to us in our Master DataFrame. We now need to look at calculating some columns.

The columns we have left to calculate are as follows:
* winner
    * The winner of the match (string)
* winner_country
    * The country of the match winner (string)
* is_international_match
    * if one or more players is outside of the UK (boolean)
* is_international_tournament
    * If the tournament was held outside of the UK (boolean)

I am going to define the UK as the following countries:
* England;
* Scotland;
* Wales;
* Ireland; and
* Northern Ireland

Any country outside of this will be considered international.

In [39]:
# Calculate match winner
df_master['match_winner'] = df_master.apply(
    lambda row: row['player1_name'] if row['player1_score'] > row['player2_score'] else row['player2_name'],
    axis=1
)

# Display head
df_master.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins


In [40]:
# Calculate Match Winner's country
df_master['match_winner_country'] = df_master.apply(
    lambda row: row['player1_country'] if row['player1_score'] > row['player2_score'] else row['player2_country'],
    axis=1
)

# Display head
df_master.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths,Wales
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins,Northern Ireland
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982.0,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins,Northern Ireland


In [ ]:
# Calculate if match is an international match
# If one or more players are not from the UK then it is considered an international match
UK_countries = ['UK', 'England', 'Scotland', 'Wales', 'Northern Ireland', 'Ireland']

# Calculate if match is international
df_master['is_international_match'] = ~df_master['player1_country'].isin(UK_countries) | ~df_master['player2_country'].isin(UK_countries)

# Calculate if tournament is international
df_master['is_international_tournament'] = ~df_master['tournament_country'].isin(UK_countries)

# Check for missing values
df_master.isnull().sum()

tournament_id                     0
match_id                          0
stage                             0
best_of                           0
player1_name                      0
player2_name                      0
player1_score                     0
player2_score                     0
season                         1088
year                            873
tournament_name                 873
tournament_country              873
tournament_prize_pool           873
tournament_status               873
player1_country                 518
player2_country                 438
match_winner                      0
match_winner_country            524
is_international_match            0
is_international_tournament       0
dtype: int64

In [42]:
# Show rows with missing values
df_master[df_master.isnull().any(axis=1)]

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country,is_international_match,is_international_tournament
2390,756,82909,Group,10,Eddie Charlton,Doug Mountjoy,6,4,NaN,NaN,NaN,NaN,NaN,NaN,Australia,Wales,Eddie Charlton,Australia,True,True
2391,756,82910,Group,10,Eddie Charlton,David Taylor,6,4,NaN,NaN,NaN,NaN,NaN,NaN,Australia,England,Eddie Charlton,Australia,True,True
2392,756,82911,Group,10,Eddie Charlton,Jimmy White,6,4,NaN,NaN,NaN,NaN,NaN,NaN,Australia,England,Eddie Charlton,Australia,True,True
2393,756,82912,Group,10,Eddie Charlton,Ray Reardon,6,4,NaN,NaN,NaN,NaN,NaN,NaN,Australia,Wales,Eddie Charlton,Australia,True,True
2394,756,82913,Group,10,Eddie Charlton,John Spencer,7,3,NaN,NaN,NaN,NaN,NaN,NaN,Australia,England,Eddie Charlton,Australia,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192733,2842,180923,Group 7,5,Patrik Tiihonen,Martin Søndergaard,3,1,2018-2019,2019.0,Nordic Snooker Championship,Sweden,0.0,Amateur,Finland,NaN,Patrik Tiihonen,Finland,True,True
192820,2797,180103,Last 16,7,Maciej Kusak,Michał Ebert,4,0,2018-2019,2019.0,Polish Amateur Championship,Poland,0.0,Amateur,Poland,NaN,Maciej Kusak,Poland,True,True
192837,2797,179957,Group 6,5,Michał Ebert,Mateusz Baranowski,3,1,2018-2019,2019.0,Polish Amateur Championship,Poland,0.0,Amateur,NaN,Poland,Michał Ebert,NaN,True,True
192838,2797,179994,Group 6,5,Tom Limor,Michał Ebert,3,2,2018-2019,2019.0,Polish Amateur Championship,Poland,0.0,Amateur,Israel,NaN,Tom Limor,Israel,True,True


In [43]:
# Filter out any matches that aren't on the professional tour
df_master = df_master[df_master['tournament_status'] == 'Professional']

In [44]:
# Check for missing values
df_master.isnull().sum()

tournament_id                   0
match_id                        0
stage                           0
best_of                         0
player1_name                    0
player2_name                    0
player1_score                   0
player2_score                   0
season                          0
year                            0
tournament_name                 0
tournament_country              0
tournament_prize_pool           0
tournament_status               0
player1_country                 6
player2_country                19
match_winner                    0
match_winner_country           10
is_international_match          0
is_international_tournament     0
dtype: int64

In [45]:
# drop rows with missing values
df_master.dropna(inplace=True)

# Check for missing values
df_master.isnull().sum()

tournament_id                  0
match_id                       0
stage                          0
best_of                        0
player1_name                   0
player2_name                   0
player1_score                  0
player2_score                  0
season                         0
year                           0
tournament_name                0
tournament_country             0
tournament_prize_pool          0
tournament_status              0
player1_country                0
player2_country                0
match_winner                   0
match_winner_country           0
is_international_match         0
is_international_tournament    0
dtype: int64

In [46]:
# Check information about the DataFrame
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102814 entries, 0 to 193529
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   tournament_id                102814 non-null  int64  
 1   match_id                     102814 non-null  int64  
 2   stage                        102814 non-null  object 
 3   best_of                      102814 non-null  int64  
 4   player1_name                 102814 non-null  object 
 5   player2_name                 102814 non-null  object 
 6   player1_score                102814 non-null  int64  
 7   player2_score                102814 non-null  int64  
 8   season                       102814 non-null  object 
 9   year                         102814 non-null  float64
 10  tournament_name              102814 non-null  object 
 11  tournament_country           102814 non-null  object 
 12  tournament_prize_pool        102814 non-null  float64
 13  tour

In [ ]:
# Convert year column to integer
df_master['year'] = df_master['year'].astype('int64')

# Convert prize column to integer
df_master['tournament_prize_pool'] = df_master['tournament_prize_pool'].round(2)

# Display master DataFrame
df_master

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country,is_international_match,is_international_tournament
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths,Wales,False,False
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins,Northern Ireland,False,False
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins,Northern Ireland,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193525,3275,203168,Group 1,5,Neil Robertson,Luca Brecel,3,1,2019-2020,2020,Championship League,England,0.0,Professional,Australia,Belgium,Neil Robertson,Australia,True,False
193526,3275,203174,Group 1,5,Neil Robertson,Jack Lisowski,3,1,2019-2020,2020,Championship League,England,0.0,Professional,Australia,England,Neil Robertson,Australia,True,False
193527,3275,203167,Group 1,5,Mark Selby,Luca Brecel,3,0,2019-2020,2020,Championship League,England,0.0,Professional,England,Belgium,Mark Selby,England,True,False
193528,3275,203169,Group 1,5,Mark Selby,Ryan Day,3,2,2019-2020,2020,Championship League,England,0.0,Professional,England,Wales,Mark Selby,England,False,False


In [ ]:
# Save master DataFrame
df_master.to_csv(os.path.join(clean_data_dir, 'master_cleaned.csv'), index=False)
print("Master DataFrame saved successfully.")

Master DataFrame saved successfully.


---

# DataFrame Filtering and Grouping

Now that we have a master file containing all the data we need, I am going to break this down into some smaller, more manageable files that will make analysis easier

In [49]:
# Load master_cleaned.csv
df_master = pd.read_csv(os.path.join(clean_data_dir, "master_cleaned.csv"))

# Display head
df_master.head()

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country,is_international_match,is_international_tournament
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths,Wales,False,False
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins,Northern Ireland,False,False
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins,Northern Ireland,False,False


### Triple Crown Events - World Championship, UK Championship, Masters

These three events are considered the gold standard of snooker. 

Players are usually measured on how many of these events they win across their career. 

In particular the World Championship is regarded as one the greatest achievements in the sport.

I am going to look at these three events separately from the rest as I believe that they have the biggest impact on the sport as a whole.

While these events are always held in the UK they have become more international as time goes on.

In [50]:
# Define Triple Crown Events
triple_crown_events = ['World Championship', 'UK Championship', 'Masters']

# Create a copy of the master DataFrame
df_triple_crown = df_master.copy()

# Filter for Triple Crown Events
df_triple_crown = df_triple_crown[df_triple_crown['tournament_name'].isin(triple_crown_events)]

# Display the filtered DataFrame
df_triple_crown

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country,is_international_match,is_international_tournament
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths,Wales,False,False
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins,Northern Ireland,False,False
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins,Northern Ireland,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102521,2821,180818,Last 16,11,Ding Junhui,Jack Lisowski,6,1,2018-2019,2019,Masters,England,590000.0,Professional,China,England,Ding Junhui,China,True,False
102522,2821,180817,Last 16,11,Ronnie O'Sullivan,Stuart Bingham,6,2,2018-2019,2019,Masters,England,590000.0,Professional,England,England,Ronnie O'Sullivan,England,False,False
102523,2821,180819,Last 16,11,Neil Robertson,Mark Williams,6,3,2018-2019,2019,Masters,England,590000.0,Professional,Australia,Wales,Neil Robertson,Australia,True,False
102524,2821,180852,Last 16,11,Mark Selby,Stephen Maguire,6,2,2018-2019,2019,Masters,England,590000.0,Professional,England,Scotland,Mark Selby,England,False,False


In [ ]:
# Save Triple Crown Events data
df_triple_crown.to_csv(os.path.join(clean_data_dir, 'triple_crown_events.csv'), index=False)
print("Triple Crown events data saved successfully.")

Triple Crown events data saved successfully.


In [ ]:
# Read triple crown events data
df_triple_crown = pd.read_csv(os.path.join(clean_data_dir, 'triple_crown_events.csv'))

# Display triple crown information
df_triple_crown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15518 entries, 0 to 15517
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   tournament_id                15518 non-null  int64  
 1   match_id                     15518 non-null  int64  
 2   stage                        15518 non-null  object 
 3   best_of                      15518 non-null  int64  
 4   player1_name                 15518 non-null  object 
 5   player2_name                 15518 non-null  object 
 6   player1_score                15518 non-null  int64  
 7   player2_score                15518 non-null  int64  
 8   season                       15518 non-null  object 
 9   year                         15518 non-null  int64  
 10  tournament_name              15518 non-null  object 
 11  tournament_country           15518 non-null  object 
 12  tournament_prize_pool        15518 non-null  float64
 13  tournament_statu

In [ ]:
# Display triple crown DataFrame
df_triple_crown

,tournament_id,match_id,stage,best_of,player1_name,player2_name,player1_score,player2_score,season,year,tournament_name,tournament_country,tournament_prize_pool,tournament_status,player1_country,player2_country,match_winner,match_winner_country,is_international_match,is_international_tournament
0,753,82716,Final,31,Terry Griffiths,Alex Higgins,16,15,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,Northern Ireland,Terry Griffiths,Wales,False,False
1,753,82718,Semi-final,17,Terry Griffiths,Tony Meo,9,7,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
2,753,82717,Semi-final,17,Alex Higgins,Ray Reardon,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,Wales,Alex Higgins,Northern Ireland,False,False
3,753,82721,Quarter-final,17,Terry Griffiths,Steve Davis,9,6,1982-1983,1982,UK Championship,England,47000.0,Professional,Wales,England,Terry Griffiths,Wales,False,False
4,753,82719,Quarter-final,17,Alex Higgins,John Spencer,9,5,1982-1983,1982,UK Championship,England,47000.0,Professional,Northern Ireland,England,Alex Higgins,Northern Ireland,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15513,2821,180818,Last 16,11,Ding Junhui,Jack Lisowski,6,1,2018-2019,2019,Masters,England,590000.0,Professional,China,England,Ding Junhui,China,True,False
15514,2821,180817,Last 16,11,Ronnie O'Sullivan,Stuart Bingham,6,2,2018-2019,2019,Masters,England,590000.0,Professional,England,England,Ronnie O'Sullivan,England,False,False
15515,2821,180819,Last 16,11,Neil Robertson,Mark Williams,6,3,2018-2019,2019,Masters,England,590000.0,Professional,Australia,Wales,Neil Robertson,Australia,True,False
15516,2821,180852,Last 16,11,Mark Selby,Stephen Maguire,6,2,2018-2019,2019,Masters,England,590000.0,Professional,England,Scotland,Mark Selby,England,False,False
